In [118]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

In [3]:
bids_df= pd.read_csv('data/bids.csv')
train_df = pd.read_csv('data/train.csv')
test_df= pd.read_csv('data/test.csv')

In [4]:
bids_df

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3
...,...,...,...,...,...,...,...,...,...
7656329,7656329,626159dd6f2228ede002d9f9340f75b7puk8d,3e64w,jewelry,phone91,9709222052631578,ru,140.204.227.63,cghhmomsaxi6pug
7656330,7656330,a318ea333ceee1ba39a494476386136a826dv,xn0y0,mobile,phone236,9709222052631578,pl,24.232.159.118,wgggpdg2gx5pesn
7656331,7656331,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone80,9709222052631578,za,80.237.28.246,5xgysg14grlersa
7656332,7656332,d4bd412590f5106b9d887a43c51b254eldo4f,hmwk8,jewelry,phone349,9709222052631578,my,91.162.27.152,bhtrek44bzi2wfl


# Feature Engineering

In [5]:
bids_unique= bids_df.groupby('bidder_id').nunique()
bids_unique = bids_unique.drop(["bid_id"],axis = 1)

#### From bids_unique dataframe, we may be interested in the number of times a bidder has 
1. Bidded in a different auction
2. Bidded from a different url
3. Bidded on a different device
4. Bidded from a different country

In [6]:
bids_unique

,auction,merchandise,device,time,country,ip,url
bidder_id,,,,,,,
001068c415025a009fee375a12cff4fcnht8y,1,1,1,1,1,1,1
002d229ffb247009810828f648afc2ef593rb,1,1,2,2,1,1,1
0030a2dd87ad2733e0873062e4f83954mkj86,1,1,1,1,1,1,1
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3,1,3,3,1,3,2
00486a11dff552c4bd7696265724ff81yeo9v,13,1,8,20,1,10,7
...,...,...,...,...,...,...,...
ffbc0fdfbf19a8a9116b68714138f2902cc13,637,1,792,23487,102,18726,8039
ffc4e2dd2cc08249f299cab46ecbfacfobmr3,15,1,13,22,6,18,12
ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1,1,1,1,1,1,1


In [7]:
bids_df_count= bids_df.groupby("bidder_id")
counts= bids_df_count['url'].count().reset_index().rename(columns = {'url':'num_bids'})

#### From counts dataframe, we know the number of times that a unique bidder has performed a bid 

In [8]:
counts

,bidder_id,num_bids
0,001068c415025a009fee375a12cff4fcnht8y,1
1,002d229ffb247009810828f648afc2ef593rb,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3
4,00486a11dff552c4bd7696265724ff81yeo9v,20
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,25075
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,22
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1
6612,ffd62646d600b759a985d45918bd6f0431vmz,664


#### Joining the new features together with the training dataset

In [67]:
new_train = train_df.merge(bids_unique, how= 'left', on = 'bidder_id')
new_train = new_train.merge(counts, how= 'left')
new_train.fillna(0,inplace = True)

new_test = test_df.merge(bids_unique, how= 'left', on = 'bidder_id')
new_test = new_test.merge(counts, how= 'left')
new_test.fillna(0,inplace = True)

#### Check that the size new training and test dataset is the same as the old training and test dataset

In [68]:
print("Size of old train: " + str(train_df.shape))
print("Size of new train: " + str(new_train.shape))
print("Size of old test: " + str(test_df.shape))
print("Size of new test: " + str(new_test.shape))

Size of old train: (2013, 4)
Size of new train: (2013, 12)
Size of old test: (4700, 3)
Size of new test: (4700, 11)


#### Count of number of first and last bids for each bidder

In [69]:
auctions = bids_df.groupby('auction')
first_bid_list = set(auctions.first()['bidder_id'])
last_bid_list = set(auctions.last()['bidder_id'])

In [70]:
# functions to add a counter if a particular bidder id is in the first and/or last bid
def check_first_bid(dummy):
    count = 0
    for bidder_id in first_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
def check_last_bid(dummy):
    count = 0
    for bidder_id in last_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count

In [71]:
num_first_bids_train = new_train['bidder_id'].apply(func = check_first_bid)
num_last_bids_train = new_train['bidder_id'].apply(func = check_last_bid)

num_first_bids_test = new_test['bidder_id'].apply(func = check_first_bid)
num_last_bids_test = new_test['bidder_id'].apply(func = check_last_bid)

In [72]:
# joining the new train and test with the number of first and last bids
new_train['num_first_bids'] = num_first_bids_train
new_train['num_last_bids'] = num_last_bids_train
new_train

,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,ip,url,num_bids,num_first_bids,num_last_bids
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,24.0,0,0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,3.0,0,0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,4.0,0,0
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,155.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,36.0,1,0
2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,0,0
2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0


In [73]:
new_test['num_first_bids'] = num_first_bids_test
new_test['num_last_bids'] = num_last_bids_test
new_test

,bidder_id,payment_account,address,auction,merchandise,device,time,country,ip,url,num_bids,num_first_bids,num_last_bids
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,a3d2de7675556553a5f08e4c88d2c228htx90,5d9fa1b71f992e7c7a106ce4b07a0a754le7c,3.0,1.0,2.0,4.0,3.0,4.0,3.0,4.0,0,0
1,a921612b85a1494456e74c09393ccb65ylp4y,a3d2de7675556553a5f08e4c88d2c228rs17i,a3d2de7675556553a5f08e4c88d2c228klidn,2.0,1.0,3.0,3.0,2.0,2.0,1.0,3.0,0,0
2,6b601e72a4d264dab9ace9d7b229b47479v6i,925381cce086b8cc9594eee1c77edf665zjpl,a3d2de7675556553a5f08e4c88d2c228aght0,14.0,1.0,4.0,17.0,3.0,4.0,2.0,17.0,0,0
3,eaf0ed0afc9689779417274b4791726cn5udi,a3d2de7675556553a5f08e4c88d2c228nclv5,b5714de1fd69d4a0d2e39d59e53fe9e15vwat,90.0,1.0,81.0,148.0,14.0,129.0,80.0,148.0,0,0
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,a3d2de7675556553a5f08e4c88d2c228dtdkd,c3b363a3c3b838d58c85acf0fc9964cb4pnfa,20.0,1.0,17.0,23.0,2.0,17.0,1.0,23.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,41.0,1.0,9.0,466.0,5.0,22.0,4.0,983.0,0,0
4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,32.0,1.0,29.0,66.0,10.0,49.0,18.0,66.0,0,0
4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,419.0,1.0,376.0,2156.0,86.0,1460.0,1049.0,2162.0,1,1
4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,5.0,1.0,4.0,5.0,1.0,5.0,2.0,5.0,0,0


In [ ]:
# stdev of time per bid (skipz first)

In [48]:
# mean response time
def find_time_to_bid(auction_id, time):
    starttime = timing_dict[auction_id]['starttime']
    return int(time) - int(starttime)

end_auction_time= bids_df.groupby('auction').time.max().reset_index()
end_auction_time= end_auction_time.rename(columns= {'time': 'endtime'})

start_auction_time= bids_df.groupby('auction').time.min().reset_index()
start_auction_time= start_auction_time.rename(columns= {'time': 'starttime'})

start_end_times= pd.merge(start_auction_time, end_auction_time, on= 'auction', how= 'left')
start_end_times = start_end_times.set_index('auction')

timing_dict = start_end_times.to_dict('index')

start_auction_time= bids_df.groupby('auction').time.min().reset_index()
start_auction_time= start_auction_time.rename(columns= {'time': 'starttime'})
timing_dict = start_end_times.to_dict('index')

bids_df['time_to_bid'] = bids_df.apply(lambda row: find_time_to_bid(row['auction'], row['time']), axis=1)

bids_df_time = bids_df
bids_df_time_mean = bids_df_time.groupby('bidder_id')['time_to_bid'].mean().reset_index()
bids_df_time_mean

,bidder_id,time_to_bid
0,001068c415025a009fee375a12cff4fcnht8y,7.439058e+13
1,002d229ffb247009810828f648afc2ef593rb,1.455263e+11
2,0030a2dd87ad2733e0873062e4f83954mkj86,7.263689e+13
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,6.854544e+12
4,00486a11dff552c4bd7696265724ff81yeo9v,1.904773e+13
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,5.719356e+12
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,2.954407e+13
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.044305e+13
6612,ffd62646d600b759a985d45918bd6f0431vmz,5.369726e+12


In [56]:
# number of simul bids per bidder
results = bids_df.groupby(['bidder_id','time']).count()[bids_df.groupby(['bidder_id','time']).count()['bid_id'] > 1 ]
inst_resp = results.groupby("bidder_id").sum()
needed_df = bids_df.groupby(['bidder_id','time']).count()[bids_df.groupby(['bidder_id','time']).count()['bid_id'] > 0 ]
num_resp = needed_df.groupby("bidder_id").sum()
num_resp = num_resp.reset_index()
inst_resp = inst_resp.reset_index()
num_resp = num_resp[['bidder_id','bid_id']]
inst_resp = inst_resp[['bidder_id','bid_id']]
num_resp.rename(columns={"bid_id": "num_bids"}, inplace= True)
inst_resp.rename(columns={"bid_id": "inst_resp"}, inplace= True)
total_inst_resp = num_resp.merge(inst_resp, how= 'left',on='bidder_id')
total_inst_resp.fillna(0, inplace = True)
total_inst_resp['perc_inst_resp'] = total_inst_resp['inst_resp']/total_inst_resp['num_bids']
total_inst_resp.drop(axis = 1 , columns = 'num_bids', inplace=True)
total_inst_resp

,bidder_id,inst_resp,perc_inst_resp
0,001068c415025a009fee375a12cff4fcnht8y,0.0,0.000000
1,002d229ffb247009810828f648afc2ef593rb,0.0,0.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,0.0,0.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,0.0,0.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,0.0,0.000000
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,3068.0,0.122353
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,0.0,0.000000
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,0.0,0.000000
6612,ffd62646d600b759a985d45918bd6f0431vmz,0.0,0.000000


In [91]:
# one-hot encoding merchandise - working
one_hot = pd.get_dummies(bids_df['merchandise'])
one_hot_df = bids_df.join(one_hot)
one_hot_df = one_hot_df.drop('merchandise', axis=1)
# one_hot_df = one_hot_df.join(one_hot)
# one_hot_df = one_hot_df.rename(columns={1.0:'merchandise=1.0',2.0:'merchandise=2.0'})
one_hot_df.set_index("bidder_id", inplace=True)
one_hot_df = one_hot_df[['auto parts', 'books and music', 'clothing', 'computers',
       'furniture', 'home goods', 'jewelry', 'mobile', 'office equipment',
       'sporting goods']]

In [ ]:
# percent instant resp (skipz first)

In [57]:
# bids per auction
bids_auction_count= bids_df.groupby(["bidder_id", "auction"])
bids_auction_count_df = bids_auction_count.nunique().groupby("bidder_id").mean().reset_index()
bids_auction_count_df = pd.DataFrame(bids_auction_count_df, columns = ['bidder_id', 'bid_id', 'auction', 'merchandise', 
                                                                       'time', 'country', 'ip', 'url'])
bids_auction_count_df = bids_auction_count_df[['bidder_id','bid_id']]
bids_auction_count_df.rename(columns={"bid_id": "num_bids_per_auction"}, inplace= True)

In [59]:
# bids per device
bids_device_count= bids_df.groupby(["bidder_id", "device"])
bids_device_count_df = bids_device_count.nunique().groupby("bidder_id").mean().reset_index()
bids_device_count_df = pd.DataFrame(bids_device_count_df, columns = ['bidder_id', 'bid_id', 'auction', 'merchandise', 
                                                                     'time', 'country', 'ip', 'url'])
bids_device_count_df = bids_device_count_df[['bidder_id','bid_id']]
bids_device_count_df.rename(columns={"bid_id": "num_bids_per_device"}, inplace= True)

In [60]:
# bids per country
bids_country_count= bids_df.groupby(["bidder_id", "country"])
bids_country_count_df = bids_country_count.nunique().groupby("bidder_id").mean().reset_index()
bids_country_count_df = pd.DataFrame(bids_country_count_df, columns = ['bidder_id', 'bid_id', 'auction', 'merchandise', 
                                                                       'time', 'country', 'ip', 'url'])
bids_country_count_df = bids_country_count_df[['bidder_id','bid_id']]
bids_country_count_df.rename(columns={"bid_id": "num_bids_per_country"}, inplace= True)

In [61]:
# bids per ip
bids_ip_count= bids_df.groupby(["bidder_id", "ip"])
bids_ip_count_df = bids_ip_count.nunique().groupby("bidder_id").mean().reset_index()
bids_ip_count_df = pd.DataFrame(bids_ip_count_df, columns = ['bidder_id', 'bid_id', 'auction', 
                                                             'merchandise', 'time', 'country', 'ip', 'url'])
bids_ip_count_df = bids_ip_count_df[['bidder_id','bid_id']]
bids_ip_count_df.rename(columns={"bid_id": "num_bids_per_ip"}, inplace= True)

In [64]:
merged_df = bids_auction_count_df.merge(bids_device_count_df, how= 'left', on = 'bidder_id')
merged_df = merged_df.merge(bids_country_count_df, how= 'left', on = 'bidder_id')
merged_df = merged_df.merge(bids_ip_count_df, how= 'left', on = 'bidder_id')
merged_df = pd.DataFrame(merged_df)
merged_df.fillna(0,inplace = True)
merged_df 

,bidder_id,num_bids_per_auction,num_bids_per_device,num_bids_per_country,num_bids_per_ip
0,001068c415025a009fee375a12cff4fcnht8y,1.000000,1.000000,1.000000,1.000000
1,002d229ffb247009810828f648afc2ef593rb,2.000000,1.000000,2.000000,2.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.000000,1.000000,1.000000,1.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.000000,1.000000,3.000000,1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,1.538462,2.500000,20.000000,2.000000
...,...,...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,39.364207,31.660354,245.833333,1.339047
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.466667,1.692308,3.666667,1.222222
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.000000,1.000000,1.000000,1.000000
6612,ffd62646d600b759a985d45918bd6f0431vmz,12.072727,6.916667,664.000000,17.945946


#### Merging all features together again

In [102]:
merged_df_2 = new_train.merge(bids_df_time_mean, how= 'left',on='bidder_id')
merged_df_2 = merged_df_2.merge(total_inst_resp, how= 'left',on='bidder_id')
merged_df_2 = merged_df_2.merge(one_hot_df, how= 'left',on='bidder_id')
merged_df_2 = merged_df_2.merge(merged_df, how= 'left',on='bidder_id')

MemoryError: Unable to allocate 3.15 TiB for an array with shape (433349045667,) and data type int64

In [103]:
new_train = merged_df_2
new_train.fillna(0,inplace = True)

In [105]:
merged_df_3 = new_test.merge(bids_df_time_mean, how= 'left',on='bidder_id')
merged_df_3 = merged_df_3.merge(total_inst_resp, how= 'left',on='bidder_id')
merged_df_3 = merged_df_3.merge(one_hot_df, how= 'left',on='bidder_id')
merged_df_3 = merged_df_3.merge(merged_df, how= 'left',on='bidder_id')
new_test = merged_df_3
new_test.fillna(0,inplace = True)

#### Splitting X and y

In [111]:
X = new_train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise'])
y = new_train['outcome']

#### Scaling the values

In [112]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.70800627, -1.46983222, -0.89443445, ..., -1.29550311,
        -0.61608126, -1.02943944],
       [-1.70800627, -1.46983222, -0.89443445, ..., -1.29550311,
        -0.61608126, -1.02943944],
       [-1.70800627, -1.46983222, -0.89443445, ..., -1.29550311,
        -0.61608126, -1.02943944],
       ...,
       [-1.74681822, -1.48593232, -0.89466119, ..., -1.36965385,
        -0.61608126, -1.02943944],
       [-1.74681822, -1.48593232, -0.89465133, ..., -1.1219052 ,
        -0.61608126, -1.02943944],
       [-1.74681822, -1.48593232, -0.89465133, ..., -1.1219052 ,
        -0.61608126, -1.02943944]])

#### Performing Stratified K-Fold with SMOTE

In [17]:
# def training_model(model, num_splits, X, y):
#     skfolds = StratifiedKFold(n_splits=num_splits)

#     for train_index, test_index in skfolds.split(X, y):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]
# #         print(y_train.value_counts())
        
#         sm = SMOTE()
#         X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
# #         print(y_train_oversampled.value_counts())
        
#         model = model
#         model.fit(X_train_oversampled, y_train_oversampled )  
#         y_pred = model.predict(X_test)
        
#         print(f'Accuracy: {model.score(X_test, y_test)}')
#         print(f'f-score: {f1_score(y_test, y_pred)}')

In [18]:
# model = XGBClassifier()
# num_splits = 10
# training_model(model, num_splits, X, y)

# Feature Selection

### Backward Elimination
Feed all possible features, then iteratively remove the worst performing features one by one till the overall performance of the model comes in acceptable range. Uses pvalue as the performance metric

In [113]:
# from this step we can see which pvalues are significant
X_1 = sm.add_constant(X)
model = sm.OLS(y, X_1).fit()
model.pvalues

C:\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:130: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


const                    0.000000e+00
auction                  0.000000e+00
device                  2.927627e-207
time                     0.000000e+00
country                  0.000000e+00
ip                       0.000000e+00
url                      0.000000e+00
num_bids                 0.000000e+00
num_first_bids           0.000000e+00
num_last_bids            3.590223e-59
time_to_bid_x            0.000000e+00
inst_resp_x              0.000000e+00
perc_inst_resp_x         0.000000e+00
auto parts              7.305903e-175
books and music         1.538468e-257
clothing                9.928432e-134
computers                0.000000e+00
furniture                0.000000e+00
home goods               0.000000e+00
jewelry                  0.000000e+00
mobile                   0.000000e+00
office equipment         0.000000e+00
sporting goods           0.000000e+00
num_bids_per_auction    8.611999e-242
num_bids_per_device      0.000000e+00
num_bids_per_country     0.000000e+00
num_bids_per

In [114]:
cols = list(X.columns)
pmax = 1 # we can adjust this

while (len(cols)>0):
    p = []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
        
selected_features_BE = cols
print(selected_features_BE)

['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids', 'num_first_bids', 'num_last_bids', 'time_to_bid_x', 'inst_resp_x', 'perc_inst_resp_x', 'auto parts', 'books and music', 'clothing', 'computers', 'furniture', 'home goods', 'jewelry', 'mobile', 'office equipment', 'sporting goods', 'num_bids_per_auction', 'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip', 'time_to_bid_y', 'inst_resp_y', 'perc_inst_resp_y']


### Recursive Feature Elimination
Recursively removing attributes and building a model on those attributes that remain. Uses accuracy as the performance metric to rank the feature according to their importance

In [115]:
nof_list = np.arange(1,11) # 11 is our maximum number of features
high_score = 0 # keeps track of the highest accuracy model

nof = 0 # returns us the ideal number of features based on the highest accuracy
score_list = []
for n in range(len(nof_list)):
    # we can modify to add smote and stratified kfold later
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0) 

    model = XGBClassifier()
    rfe = RFE(model, nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)

    model.fit(X_train_rfe, y_train)
    score = model.score(X_test_rfe, y_test)
    score_list.append(score)
    if score > high_score:
        high_score = score
#         nof = nof_list[n]
#         for i in range(X.shape[1]):
#             print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support[i], rfe.ranking_[i]))
        
nof

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:20:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:22:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:00:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:01:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


AttributeError: 'RFE' object has no attribute 'support'

### Embedded Method
Takes care of each iteration of the model training process and carefully extract features which contribute the most to the training for a particular iteration. Uses regularization methods to penalize a feature given a coefficient threshold

In [121]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = LassoCV(alphas=np.arange(0.1, 10, 0.1), cv=cv, n_jobs=-1)

model.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92427.73029341936, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92448.86884476311, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92415.71942512254, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91413.0657484926, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91467.7909693695, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91519.54704929286, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might w

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92700.30709683268, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92634.35997741454, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92567.27739412957, tolerance: 32.1234542527325
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 89936.31231285466, tolerance: 32.14122652149702
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 90732.55049541782, tolerance: 32.134645833905815
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 89529.44219678815, tolerance: 32.15183041307749
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You m

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 86113.04005411958, tolerance: 32.137864014254006
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91532.31975549231, tolerance: 32.134645833905815
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 90270.96839143417, tolerance: 32.120235445210206
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You

...

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91855.32987493204, tolerance: 32.13888795136124
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93473.52046788171, tolerance: 32.13618180110346
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91890.84000654657, tolerance: 32.13888795136124
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You mi

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92110.33255304082, tolerance: 32.13888795136124
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92037.75220142983, tolerance: 32.13888795136124
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92817.73230936332, tolerance: 32.130257180415704
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You m

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91390.29522848176, tolerance: 32.130257180415704
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91457.92849793388, tolerance: 32.130257180415704
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91521.82131873256, tolerance: 32.130257180415704
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91093.96121576054, tolerance: 32.130257180415704
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 90746.77233451308, tolerance: 32.130257180415704
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 90178.76237336964, tolerance: 32.130257180415704
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You

MemoryError: Unable to allocate 612. MiB for an array with shape (2764127, 29) and data type float64